In [ ]:
import pandas as pd

from prophet import Prophet

df = pd.read_csv(r'/Users/apple/Desktop/Barrior/newCSV/us_weekly_consumable_water_summary_20220102-20230903_2.csv')

df.columns = ['ds', 'y']
df.head()